# Whisper GPU Transcription on Google Colab

이 노트북을 Google Colab에서 실행하세요:
1. 이 파일을 Google Colab에 업로드
2. 런타임 → 런타임 유형 변경 → GPU 선택 (T4 추천)
3. 아래 셀들을 순서대로 실행

In [ ]:
# GPU 확인
!nvidia-smi

In [ ]:
# Whisper 설치
!pip install -q openai-whisper

In [ ]:
# Google Drive 마운트 (파일 업로드/다운로드용)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 파일 직접 업로드 (작은 파일용)
from google.colab import files
uploaded = files.upload()

# 업로드된 파일명 확인
audio_file = list(uploaded.keys())[0]
print(f"업로드된 파일: {audio_file}")

In [ ]:
# Whisper 실행 함수
import whisper
import time
import torch

def transcribe_audio(audio_path, model_name="base"):
    # GPU 사용 확인
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    # 모델 로드
    print(f"Loading {model_name} model...")
    model = whisper.load_model(model_name, device=device)
    
    # 전사 시작
    print(f"\nTranscribing {audio_path}...")
    start_time = time.time()
    
    result = model.transcribe(
        audio_path,
        language="ko",  # 한국어 지정 (더 빠름)
        fp16=True,      # GPU에서 FP16 사용 (더 빠름)
        verbose=True
    )
    
    elapsed_time = time.time() - start_time
    print(f"\n전사 완료! 소요 시간: {elapsed_time:.1f}초")
    
    return result

In [ ]:
# 전사 실행 (모델 선택: tiny, base, small, medium, large, large-v3)
result = transcribe_audio(audio_file, model_name="base")

# 결과 확인
print(f"\n언어: {result['language']}")
print(f"텍스트 길이: {len(result['text'])} 글자")
print(f"\n전사 결과 (처음 500자):\n{result['text'][:500]}...")

In [ ]:
# 결과 저장
output_filename = audio_file.rsplit('.', 1)[0] + '_transcript.txt'

# 텍스트 파일로 저장
with open(output_filename, 'w', encoding='utf-8') as f:
    f.write(result['text'])

# SRT 자막 파일 생성
def write_srt(result, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for i, segment in enumerate(result['segments'], 1):
            f.write(f"{i}\n")
            f.write(f"{format_timestamp(segment['start'])} --> {format_timestamp(segment['end'])}\n")
            f.write(f"{segment['text'].strip()}\n\n")

def format_timestamp(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d},{millis:03d}"

srt_filename = audio_file.rsplit('.', 1)[0] + '.srt'
write_srt(result, srt_filename)

print(f"저장 완료:")
print(f"- 텍스트: {output_filename}")
print(f"- 자막: {srt_filename}")

In [ ]:
# 파일 다운로드
files.download(output_filename)
files.download(srt_filename)

In [ ]:
# (선택) Google Drive에 저장
import shutil

# Drive의 원하는 폴더로 복사
drive_folder = '/content/drive/MyDrive/whisper_results/'
!mkdir -p {drive_folder}

shutil.copy(output_filename, drive_folder)
shutil.copy(srt_filename, drive_folder)

print(f"Google Drive에 저장 완료: {drive_folder}")

## 배치 처리 (여러 파일 한번에)

Google Drive에 여러 오디오 파일이 있을 때 사용하세요.

In [ ]:
import os
import glob

# Drive의 오디오 폴더 지정
audio_folder = '/content/drive/MyDrive/audio_files/'
output_folder = '/content/drive/MyDrive/whisper_results/'

# 오디오 파일 찾기
audio_files = glob.glob(os.path.join(audio_folder, '*.m4a')) + \
              glob.glob(os.path.join(audio_folder, '*.mp3')) + \
              glob.glob(os.path.join(audio_folder, '*.wav'))

print(f"찾은 파일 {len(audio_files)}개:")
for f in audio_files:
    print(f"  - {os.path.basename(f)}")

# 모든 파일 처리
for audio_file in audio_files:
    print(f"\n{'='*50}")
    print(f"처리 중: {os.path.basename(audio_file)}")
    
    try:
        result = transcribe_audio(audio_file, model_name="base")
        
        # 결과 저장
        base_name = os.path.basename(audio_file).rsplit('.', 1)[0]
        txt_path = os.path.join(output_folder, base_name + '_transcript.txt')
        
        with open(txt_path, 'w', encoding='utf-8') as f:
            f.write(result['text'])
        
        print(f"✓ 저장 완료: {txt_path}")
        
    except Exception as e:
        print(f"✗ 오류 발생: {e}")